## Importing libraries


In [27]:
import gc  # garbage collector
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pylab as pylab
import optuna

pd.set_option("display.max_columns", None)

import warnings

warnings.filterwarnings("ignore")

from helper import utility as ut
import importlib

importlib.reload(ut)

from catboost import CatBoostRegressor, Pool
import lightgbm as lgb

## Load Dataset - Common for both Models


In [16]:
prop_2016 = ut.load_properties_data("clean_data/prop_2016_clean.csv")
prop_2017 = ut.load_properties_data("clean_data/prop_2017_clean.csv")
train = ut.load_properties_data("clean_data/train_combined.csv")

train.dtypes

parcelid                       int64
logerror                     float64
airconditioningtypeid          int64
architecturalstyletypeid       int64
basementsqft                 float64
                              ...   
avg_area_per_room            float64
derived_avg_area_per_room    float64
year                           int64
month                          int64
quarter                        int64
Length: 71, dtype: object

## Catboost Model


In [17]:
# Dropping columns which do not perform well when we input to the catboost model
catboost_features = ut.drop_features(train)
print("Number of features for CatBoost: {}".format(len(catboost_features.columns)))
catboost_features.head(5)

# Prepare feature list for catboost model
categorical_features = [
    "airconditioningtypeid",
    "heatingorsystemtypeid",
    "propertylandusetypeid",
    "year",
    "month",
    "quarter",
    "buildingclasstypeid",
]
for col in catboost_features.columns:
    if col in categorical_features:
        catboost_features[col] = catboost_features[col].astype("str")

Number of features for CatBoost: 55


In [18]:
# Prepare training and cross-validation data
catboost_label = train.logerror.astype(np.float32)
print(catboost_label.head())

# Transform to Numpy matrices
catboost_X = catboost_features.values
catboost_y = catboost_label.values

# Perform shuffled train/test split
X_train, X_val, y_train, y_val = train_test_split(
    catboost_X, catboost_y, test_size=0.2, random_state=99
)
ut.remove_outliers(X_train, y_train)

0    0.0276
1   -0.1684
2   -0.0040
3    0.0218
4   -0.0050
Name: logerror, dtype: float32
new_X: (131462, 55)
new_y: (131462,)


(array([['-1', 2.5, 3.0, ..., '0', '5', '2'],
        ['0', 2.0, 4.0, ..., '1', '6', '2'],
        ['0', 3.0, 4.0, ..., '0', '8', '3'],
        ...,
        ['-1', 3.0, 4.0, ..., '1', '9', '3'],
        ['-1', 2.5, 3.0, ..., '0', '4', '2'],
        ['0', 2.0, 4.0, ..., '1', '8', '3']], dtype=object),
 array([-0.001     , -0.02973739,  0.0315    , ..., -0.00443901,
         0.007     ,  0.02685545], dtype=float32))

In [19]:
# Specify feature names and categorical features for CatBoost
categorical_indices = ut.get_categorical_indices(
    catboost_features, categorical_features
)

[0, 13, 21, 52, 53, 54]


In [20]:
n = [i * 0.005 for i in range(1,int(1/0.005) + 1)]

# Printing the generated list
print(len(n))

200


In [21]:
ut.remove_outliers(catboost_X, catboost_y)


model = CatBoostRegressor(
        loss_function="MAE",
        eval_metric="MAE",
        nan_mode="Min",
        random_seed=99,
        iterations=1000,
        learning_rate=0.015,
        border_count=254,
        max_depth=6,
        random_strength=1,
        l2_leaf_reg=5,
        bagging_temperature=1,
        verbose=True,
    )
model.fit(catboost_X, catboost_y, cat_features=categorical_indices, verbose=False)

# Sanity check: score on a small portion of the dataset
print("sanity check score: {}".format(abs(model.predict(X_val) - y_val).mean()))


new_X: (164299, 55)
new_y: (164299,)
sanity check score: 0.06857669115730533


In [22]:
file_name = "submission/final_catboost_single.csv"
submission, pred_2016, pred_2017 = ut.predict_and_generate_csv(
    [model], prop_2016, prop_2017, file_name
)

Start model 0 (2016)
Start model 0 (2017)
Length of submission DataFrame: 2985217
Submission header:
   ParcelId  201610  201611  201612  201710  201711  201712
0  10754147 -0.0130 -0.0130 -0.0130 -0.0159 -0.0159 -0.0159
1  10759547 -0.0121 -0.0121 -0.0121 -0.0121 -0.0121 -0.0121
2  10843547  0.0040  0.0040  0.0040  0.0075  0.0075  0.0075
3  10859147  0.0277  0.0277  0.0277  0.0293  0.0293  0.0293
4  10879947  0.0105  0.0105  0.0105  0.0100  0.0100  0.0100


### Optuna for CatBoost Regressor

In [28]:
def objective(trial):
    # Define hyperparameters to be tuned
    params = {
        'iterations': trial.suggest_int('iterations', 10, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 4, 10),
    }

    model = CatBoostRegressor(**params, random_state=99, verbose=0)


    model.fit(X_train, y_train)

    predictions = model.predict(X_val)

    mse = mean_squared_error(y_val, predictions)

    print(f"Iteration {objective.iteration}: MSE = {mse}")
    objective.iteration += 1

    return mse


objective.iteration = 1
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best Hyperparameters:")
print(best_params)


[I 2023-11-17 15:50:33,703] A new study created in memory with name: no-name-d9aeb5b6-c00d-4d02-8bce-97c7f782c6fb
[I 2023-11-17 15:50:37,116] Trial 0 finished with value: 0.028460523013205168 and parameters: {'iterations': 222, 'learning_rate': 0.013833811467916934, 'depth': 9, 'l2_leaf_reg': 23.933481820332084, 'border_count': 34}. Best is trial 0 with value: 0.028460523013205168.


Iteration 1: MSE = 0.028460523013205168


[I 2023-11-17 15:50:38,617] Trial 1 finished with value: 0.028703450130051607 and parameters: {'iterations': 56, 'learning_rate': 0.018497443241885625, 'depth': 5, 'l2_leaf_reg': 60.21798415887893, 'border_count': 184}. Best is trial 0 with value: 0.028460523013205168.


Iteration 2: MSE = 0.028703450130051607


[I 2023-11-17 15:50:42,839] Trial 2 finished with value: 0.02810377943391018 and parameters: {'iterations': 193, 'learning_rate': 0.1989292856510661, 'depth': 9, 'l2_leaf_reg': 3.973009129260253, 'border_count': 174}. Best is trial 2 with value: 0.02810377943391018.


...

Iteration 48: MSE = 0.02808365575469817


[I 2023-11-17 16:08:26,663] Trial 48 finished with value: 0.028107633000633184 and parameters: {'iterations': 821, 'learning_rate': 0.10466350976537353, 'depth': 4, 'l2_leaf_reg': 80.33277618296705, 'border_count': 113}. Best is trial 23 with value: 0.028022388899216034.


Iteration 49: MSE = 0.028107633000633184


[I 2023-11-17 16:08:32,640] Trial 49 finished with value: 0.02808517140335897 and parameters: {'iterations': 739, 'learning_rate': 0.1472416360702783, 'depth': 5, 'l2_leaf_reg': 45.33447611896309, 'border_count': 235}. Best is trial 23 with value: 0.028022388899216034.


Iteration 50: MSE = 0.02808517140335897
Best Hyperparameters:
{'iterations': 866, 'learning_rate': 0.12398893893868002, 'depth': 10, 'l2_leaf_reg': 80.11420632515885, 'border_count': 186}


### Catboost with 4x ensemble


In [23]:
# Train multiple models
rounds = 8
models = []
for i in range(rounds):
    print("Start training model {}".format(i))
    model = CatBoostRegressor(
        loss_function="MAE",
        eval_metric="MAE",
        nan_mode="Min",
        random_seed=99,
        iterations=1000,
        learning_rate=0.015,
        border_count=254,
        max_depth=6,
        random_strength=1,
        l2_leaf_reg=5,
        bagging_temperature=1,
        verbose=True,
    )
    model.fit(catboost_X, catboost_y, cat_features=categorical_indices, verbose=True)
    models.append(model)

Start training model 0
0:	learn: 0.0688501	total: 101ms	remaining: 1m 40s
1:	learn: 0.0688426	total: 196ms	remaining: 1m 37s
2:	learn: 0.0688354	total: 286ms	remaining: 1m 34s
3:	learn: 0.0688278	total: 373ms	remaining: 1m 32s
4:	learn: 0.0688196	total: 472ms	remaining: 1m 33s
5:	learn: 0.0688108	total: 551ms	remaining: 1m 31s
6:	learn: 0.0688048	total: 651ms	remaining: 1m 32s
7:	learn: 0.0687976	total: 738ms	remaining: 1m 31s
8:	learn: 0.0687910	total: 833ms	remaining: 1m 31s
9:	learn: 0.0687832	total: 930ms	remaining: 1m 32s
10:	learn: 0.0687755	total: 1.02s	remaining: 1m 32s
11:	learn: 0.0687703	total: 1.13s	remaining: 1m 32s
12:	learn: 0.0687629	total: 1.23s	remaining: 1m 33s
13:	learn: 0.0687561	total: 1.33s	remaining: 1m 33s
14:	learn: 0.0687495	total: 1.43s	remaining: 1m 34s
15:	learn: 0.0687423	total: 1.53s	remaining: 1m 34s
16:	learn: 0.0687360	total: 1.64s	remaining: 1m 34s
17:	learn: 0.0687310	total: 1.76s	remaining: 1m 35s
18:	learn: 0.0687249	total: 1.92s	remaining: 1m 39s

KeyboardInterrupt: 

In [ ]:
# Make predictions and export results
file_name = 'submission/final_catboost_ensemble_x4.csv'
submission, pred_2016, pred_2017 = ut.predict_and_generate_csv(models, prop_2016, prop_2017, file_name)

## Lightgbm Model


In [ ]:
#Drop not useful columns
lightgbm_features = ut.drop_features(train)
print("Number of features for Lightgbm: {}".format(len(lightgbm_features.columns)))
lightgbm_features.head(5)

In [ ]:
# Prepare training and cross-validation data
lightgbm_label = train.logerror.astype(np.float32)
print(lightgbm_label.head())

# Transform to Numpy matrices
lightgbm_X = lightgbm_features.values
lightgbm_y = lightgbm_label.values

# Perform shuffled train/test split
np.random.seed(42)
random.seed(10)
X_train, X_val, y_train, y_val = train_test_split(lightgbm_X, lightgbm_y, test_size=0.2 , random_state=99)

a,b=ut.remove_outliers(X_train, y_train)
X_train=a
y_train=b

print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_val shape: {}".format(X_val.shape))
print("y_val shape: {}".format(y_val.shape))

In [ ]:
# Specify feature names and categorical features for Lightgbm
categorical_features = ['airconditioningtypeid', 'heatingorsystemtypeid', 'propertylandusetypeid', 'year', 'month', 'quarter','buildingclasstypeid']
categorical_indices = ut.get_categorical_indices(lightgbm_features, categorical_features)

In [ ]:
# Lightgbm parameters
params = {}

params["objective"] = "regression"
params["metric"] = "mae"
params["num_threads"] = 4  # set to number of real CPU cores for best performance

params["boosting_type"] = "gbdt"
params["num_boost_round"] = 1250
params["learning_rate"] = 0.003  # shrinkage_rate 
# params["early_stopping_rounds"] = 30  # Early stopping based on validation set performance 

# Control tree growing
params["num_leaves"] = 127  # max number of leaves in one tree (default 31)
params["min_data"] = 150  # min_data_in_leaf
params["min_hessian"] = 0.001  # min_sum_hessian_in_leaf (default 1e-3)
params["max_depth"] = -1  # limit the max depth of tree model, defult -1 (no limit)
params[
    "max_bin"
] = 255  # max number of bins that feature values are bucketed in (small -> less overfitting, default 255)
params[
    "sub_feature"
] = 0.5  # feature_fraction (small values => use very different submodels)

# Row subsampling (speed up training and alleviate overfitting)
params["bagging_fraction"] = 0.7
params["bagging_freq"] = 50  # perform bagging at every k iteration

# Constraints on categorical features
params[
    "min_data_per_group"
] = 100  # minimal number of data per categorical group (default 100)
params[
    "cat_smooth"
] = 15.0  # reduce effect of noises in categorical features, especially for those with few data (default 10.0)

# Regularization (default 0.0)
params["lambda_l1"] = 0.0
params["lambda_l2"] = 0.0

# Random seeds (keep default values)
params["feature_fraction_seed"] = 2
params["bagging_seed"] = 3

### Lightgbm Single Model


In [ ]:
#Train Lightgbm
feature_names = [s for s in lightgbm_features.columns]
lgb_train_set = lgb.Dataset(X_train, label=y_train, feature_name=feature_names)
lgb_valid_set = lgb.Dataset(X_val, label=y_val, feature_name=feature_names)

np.random.seed(42)
random.seed(36)
model = lgb.train(params, lgb_train_set,
                valid_sets=[lgb_train_set, lgb_valid_set], valid_names=['train', 'val'],
                categorical_feature=categorical_indices)

# Evaluate on train and validation sets
print("Train score: {}".format(abs(model.predict(X_train) - y_train).mean() * 100))
print("Val score: {}".format(abs(model.predict(X_val) - y_val).mean() * 100))

In [ ]:
# Plot LightGBM feature importance
lgb.plot_importance(model, height=0.8, figsize=(12.5, 12.5), ignore_zero=False)

In [ ]:
# Train LightGBM on all given training data (preparing for submission)
#del params['early_stopping_rounds']

a,b=ut.remove_outliers(lightgbm_X,lightgbm_y)
lightgbm_X=a
lightgbm_y=b

lgb_train_set = lgb.Dataset(lightgbm_X, label=lightgbm_y, feature_name=feature_names)
print("lightgbm_X: {}".format(lightgbm_X.shape))
print("lightgbm_y: {}".format(lightgbm_y.shape))

np.random.seed(42)
random.seed(36)
model = lgb.train(params, lgb_train_set, categorical_feature=categorical_indices)

# Sanity check: make sure the model score is reasonable on a small portion of the data
print("score: {}".format(abs(model.predict(X_val) - y_val).mean() * 100))

In [ ]:
file_name = 'submission/final_lgb_single.csv'
submission, pred_2016, pred_2017 = ut.predict_and_generate_csv([model], prop_2016, prop_2017, file_name)

### Lightgbm Ensemble 5x Model


In [ ]:
a,b=ut.remove_outliers(lightgbm_X,lightgbm_y)
lightgbm_X=a
lightgbm_y=b

lgb_train_set = lgb.Dataset(lightgbm_X, label=lightgbm_y, feature_name=feature_names)

# Train multiple models
bags = 5
models = []
for i in range(bags):
    print("Start training model {}".format(i))
    params["seed"] = i
    np.random.seed(42)
    random.seed(36)
    model = lgb.train(params, lgb_train_set, categorical_feature=categorical_indices)
    models.append(model)

# Sanity check (make sure scores on a small portion of the dataset are reasonable)
for i, model in enumerate(models):
    print("model {}: {}".format(i, abs(model.predict(X_val) - y_val).mean() * 100))

# Save the trained models to disk
ut.save_models(models,'lightgbm')

models = ut.load_lightgbm_models(['checkpoints/lightgbm_' + str(i) for i in range(bags)])  # load pretrained models 

In [ ]:
# Make predictions and export results
file_name = 'submission/final_lgb_ensemble_x5.csv'
submission, pred_2016, pred_2017 = ut.predict_and_generate_csv(models, prop_2016, prop_2017, file_name)

## Stacking


In [ ]:
lgb_single = pd.read_csv('submission/final_lgb_single.csv')
catboost_single = pd.read_csv('submission/final_catboost_single.csv')
print("Finished Loading the prediction results.")

def decimal_range(start, stop, increment):
    while start <= stop: 
        yield start
        start += increment

for weight in decimal_range(0.1, 0.9, 0.1):
    #weight = 0.7
    stack = pd.DataFrame()
    stack["ParcelId"] = lgb_single["ParcelId"]
    for col in ["201610", "201611", "201612", "201710", "201711", "201712"]:
        stack[col] = weight * catboost_single[col] + (1 - weight) * lgb_single[col]

    print(stack.head())
    #stack.to_csv("submission/final_stack.csv", index=False)
    stack.to_csv("submission/final_stack_catboostweight_" + str("{:.1f}".format(weight)) + ".csv", index=False)